In [1]:
import numpy as np
import sched, time, csv, os, StringIO

In [4]:
def downsample(files, outfile, R):
    def csv_string(data):
        si = StringIO.StringIO()
        cw = csv.writer(si, delimiter=',')
        cw.writerow(data)
        return str(si.getvalue())

    def check(output, o):
        output = [str(x) for x in output]
        rewind = o.tell()
        outline = o.readline()
        if not outline:
            o.seek(rewind)
            o.write(csv_string(output))
            print str(o.tell())+' new line added'
        else:
            outrow = list(csv.reader(StringIO.StringIO(outline), delimiter=','))[0]
            if not outrow == output:
                o.seek(rewind)
                o.write(csv_string(output))
                print str(o.tell())+' line overwritten:'
                print 'old: '+str(outrow)
                print 'new: '+str(outline)
                print

        # with open(outfile, 'wb') as f:
        # 	spamwriter = csv.writer(f, delimiter=',',
        # 							quotechar='|', quoting=csv.QUOTE_MINIMAL)
        # 	spamwriter.writerow(['date','mood','tweets'])
        for fn in files:
            prev_time = 0
            mood = []
            counter = 0
            # with open(fn, 'rb') as f:
            # 	spamreader = csv.reader(f, delimiter=',', quotechar='|')
            # 	for row in spamreader:
            o = file(outfile, 'r+')
            if not o.readline() == 'date,mood,tweets':
                o.seek(0)
                o.write(csv_string(['date','mood','tweets']))
            f = file(fn, 'r')
            f.seek(0,2)
            lastline = f.tell()
            f.seek(0)
            tweets = 0
            min_tweets = 100
            while f.tell() < lastline:
                where = f.tell()
                line = f.readline()
                if not line:
                    time.sleep(0.1)
                    f.seek(where)
                else:
                    tweets += 1
                    a = StringIO.StringIO(line)
                    reader = csv.reader(a, delimiter=',')
                    row = list(reader)[0]
                    mood.append(float(row[1]))
                    if prev_time == 0:
                        prev_time = float(row[0])
                        continue
                    time_diff = float(row[0]) - prev_time
                    if time_diff > R:
                        if int(time_diff / R) > 1: # if two bins or more passed since last entry, we're missing data
                            output = [float(row[0])-(time_diff*0.5), None, tweets]
                            check(output, o)
                        # 	# with open(outfile, 'ab') as outf:
                        # 	# 	spamwriter = csv.writer(outf, delimiter=',',
                        # 	# 							quotechar='|', quoting=csv.QUOTE_MINIMAL)
                        # 	# 	spamwriter.writerow([float(row[0])-(time_diff*0.5), None, tweets])
                        if time_diff > R and tweets > min_tweets:
                            output = [float(row[0])-(R*0.5), np.mean(mood), tweets]
                            check(output, o)
                            # with open(outfile, 'ab') as outf:
                            # 	spamwriter = csv.writer(outf, delimiter=',',
                            # 							quotechar='|', quoting=csv.QUOTE_MINIMAL)
                            # 	spamwriter.writerow([float(row[0])-(R*0.5), np.mean(mood), tweets])
                        mood = []
                        prev_time = float(row[0])
                        tweets = 0
            o.close()
            f.close()
    
def do_something(R):
    files = os.listdir('.')
    files = [fn for fn in files if fn.endswith('.csv')]

    files_sanders = sorted([fn for fn in files if 'data_sanders' in fn and not 'downsampled' in fn])
    files_trump = sorted([fn for fn in files if 'data_trump' in fn and not 'downsampled' in fn])
    files_clinton = sorted([fn for fn in files if 'data_clinton' in fn and not 'downsampled' in fn])
    files_cruz = sorted([fn for fn in files if 'data_cruz' in fn and not 'downsampled' in fn])

    downsample(files_sanders, './data_sanders_downsampled.csv', R)
    downsample(files_trump, './data_trump_downsampled.csv', R)
    downsample(files_clinton, './data_clinton_downsampled.csv', R)
    downsample(files_cruz, './data_cruz_downsampled.csv', R)

In [5]:
R = 60*60 # bin size in seconds
do_something(R)

# --- below here is old stuff ---

# Downsample per x minutes

In [28]:
minutes = 10
R = minutes * 60

In [34]:
for fn in files:
    with open('d3js/'+fn, 'wb') as f:
        spamwriter = csv.writer(f, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerow(['date','mood'])
    prev_time = 0
    mood = []
    counter = 0
    with open(fn, 'rb') as f:
        spamreader = csv.reader(f, delimiter=' ', quotechar='|')
        for row in spamreader:
            mood.append(float(row[1]))
            if prev_time == 0:
                prev_time = float(row[0])
                continue
            if float(row[0]) - prev_time > R:
                with open('d3js/'+fn, 'ab') as f:
                    spamwriter = csv.writer(f, delimiter=',',
                                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
                    spamwriter.writerow([float(row[0])-60, np.mean(mood)])
                mood = []
                prev_time = float(row[0])

# Compression

In [ ]:
import gzip
import shutil
with open('data_trump.csv', 'rb') as f_in, gzip.open('data_trump.csv.gz', 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)

# automatic file loading

In [2]:
import os

In [5]:
def downsample(files, outfile, R):
    with open(outfile, 'wb') as f:
        spamwriter = csv.writer(f, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerow(['date','mood'])
    for fn in files:
        prev_time = 0
        mood = []
        counter = 0
        with open(fn, 'rb') as f:
            spamreader = csv.reader(f, delimiter=',', quotechar='|')
            for row in spamreader:
                mood.append(float(row[1]))
                if prev_time == 0:
                    prev_time = float(row[0])
                    continue
                if float(row[0]) - prev_time > R:
                    with open(outfile, 'ab') as f:
                        spamwriter = csv.writer(f, delimiter=',',
                                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
                        spamwriter.writerow([float(row[0])-60, np.mean(mood)])
                    mood = []
                    prev_time = float(row[0])

In [7]:
files = os.listdir('.')
files = [fn for fn in files if fn.endswith('.csv')]

files_sanders = sorted([fn for fn in files if 'data_sanders' in fn])
print files_sanders
files_trump = sorted([fn for fn in files if 'data_trump' in fn])
print files_trump
files_clinton = sorted([fn for fn in files if 'data_clinton' in fn])
print files_clinton
files_cruz = sorted([fn for fn in files if 'data_cruz' in fn])
print files_cruz

R = 10 * 60 # 10 minute bins

downsample(files_sanders, 'd3js/data_sanders_downsampled.csv', R)
downsample(files_trump, 'd3js/data_trump_downsampled.csv', R)
downsample(files_clinton, 'd3js/data_clinton_downsampled.csv', R)
downsample(files_cruz, 'd3js/data_cruz_downsampled.csv', R)

['data_sanders_00000.csv']
['data_trump_00000.csv']
['data_clinton_00000.csv']
['data_cruz_00000.csv']


## test non-blocking reading

In [8]:
f = file('data_sanders_00000.csv', 'r')

In [ ]:
while 1:
    where = f.tell()
    line = f.readline()
    if not line:
        time.sleep(1)
        f.seek(where)
    else:
        print line, # already has newline

In [11]:
f.seek(0,2)
print f.tell()

3378880


In [12]:
f.seek(0)

In [25]:
import StringIO

In [27]:
x = StringIO.StringIO(f.readline())
reader = csv.reader(x, delimiter=',')
for row in reader:
    print row

['1458999471.343462', '0.0772']
